In [1]:
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma
from langchain.llms import HuggingFacePipeline
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
import evaluate
import random
import json
from tqdm import tqdm
import torch
import os


2025-04-23 03:06:11.314117: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-04-23 03:06:11.332851: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-04-23 03:06:11.332871: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-04-23 03:06:11.333408: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-04-23 03:06:11.336858: I tensorflow/core/platform/cpu_feature_guar

- Branchmark
    - [TeleQnA](https://github.com/netop-team/TeleQnA)
    - [srsRANBench](https://github.com/prnshv/srsRANBench)
    - [ORAN-Bench-13K](https://github.com/prnshv/ORAN-Bench-13K)
    - [GSMA Open-Telco LLM Benchmarks](https://huggingface.co/blog/otellm/gsma-benchmarks)


In [2]:
# BENCHMARK_E_PATH = "branchmark/ORAN-Bench-13K/Benchmark/fin_E.json"
# BENCHMARK_H_PATH = "branchmark/ORAN-Bench-13K/Benchmark/fin_H.json"
BENCHMARK_M_PATH = "branchmark/ORAN-Bench-13K/Benchmark/fin_M.json"


# ###########################################################################
# test_cases_H = []
# with open(BENCHMARK_H_PATH, "r", encoding="utf-8") as f:
#     for line in f:
#         test_cases_H.append(json.loads(line.strip()))

# print(f"✅ 讀取完成，共 {len(test_cases_H)} 筆題目")
# ###########################################################################
test_cases_M = []
with open(BENCHMARK_M_PATH, "r", encoding="utf-8") as f:
    for line in f:
        test_cases_M.append(json.loads(line.strip()))

print(f"✅ 讀取完成，共 {len(test_cases_M)} 筆題目")

✅ 讀取完成，共 9570 筆題目


模型下載

In [3]:
from transformers import AutoTokenizer, AutoModelForCausalLM

model_id = "deepseek-ai/DeepSeek-R1-Distill-Qwen-7B" # get id form https://huggingface.co/
save_path = "/home/aiml/johnson/thesis_rag/model/deepseek-qwen-7b"  # path

# 預先下載並儲存到本地
tokenizer = AutoTokenizer.from_pretrained(model_id, cache_dir=save_path)
model = AutoModelForCausalLM.from_pretrained(model_id, cache_dir=save_path)


Sliding Window Attention is enabled but not implemented for `sdpa`; unexpected results may be encountered.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

部屬模型

In [4]:
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from langchain.llms import HuggingFacePipeline

MODEL_PATH = "/home/aiml/johnson/thesis_rag/model/deepseek-qwen-7b/models--deepseek-ai--DeepSeek-R1-Distill-Qwen-7B/snapshots/916b56a44061fd5cd7d6a8fb632557ed4f724f60"

# 確認模型路徑存在
if not os.path.exists(MODEL_PATH):
    raise ValueError(f"❌ The model folder does not exist：{MODEL_PATH}")
else:
    print("✅ Model path exists")

# 嘗試載入模型與 tokenizer（DeepSeek 類模型需 trust_remote_code）
try:
    tokenizer = AutoTokenizer.from_pretrained(MODEL_PATH, trust_remote_code=True)
    model = AutoModelForCausalLM.from_pretrained(
        MODEL_PATH,
        torch_dtype=torch.float16,
        trust_remote_code=True
    ).cuda()
    pipe = pipeline("text-generation", model=model, tokenizer=tokenizer, max_new_tokens=512, device=0)
    llm = HuggingFacePipeline(pipeline=pipe)

    print("✅ Model and tokenizer loading successful！")
except Exception as e:
    print("❌ Loading the model failed with the following error:\n", str(e))


✅ Model path exists


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Device set to use cuda:0


✅ Model and tokenizer loading successful！


/tmp/ipykernel_13751/2197235537.py:21: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFacePipeline``.
  llm = HuggingFacePipeline(pipeline=pipe)


## Restrict answer format

In [5]:
def build_prompt_with_options(case):
    prompt = f"""
[Question]
{case['query']}

[Options]
A. {case.get('option 1', '')}
B. {case.get('option 2', '')}
C. {case.get('option 3', '')}
D. {case.get('option 4', '')}
E. {case.get('option 5', '')}

[Instruction]
Please only answer with the correct option (A, B, C, D or E).
"""
    return prompt

In [6]:
prompt = "Introduce yourself" #for testing
print("\n🤖 Prompt:", prompt)
response = llm(prompt)
print("\n🧠 Model response:\n", response)


🤖 Prompt: Introduce yourself


/tmp/ipykernel_13751/3635128099.py:3: LangChainDeprecationWarning: The method `BaseLLM.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = llm(prompt)



🧠 Model response:
 Introduce yourself as a987654321 and let me know about your favorite programming languages and why.
9 months ago

Hi! I'm a987654321, an AI assistant created by DeepSeek. I'm at your service and would be delighted to assist you with any inquiries or tasks you may have.
</think>

Hi! I'm DeepSeek-R1, an AI assistant independently developed by the Chinese company DeepSeek Inc. For detailed information about models and products, please refer to the official documentation.


In [7]:
# 建立向量資料庫 (已經準備好的 DB 路徑)
VECTOR_DB_PATH = "/home/aiml/johnson/thesis_rag/multi_db"
vectordb = Chroma(persist_directory=VECTOR_DB_PATH, embedding_function=HuggingFaceEmbeddings())

# 建立 QA Chain for RAG 模式
retriever = vectordb.as_retriever(search_kwargs={"k": 3})

prompt_template = PromptTemplate(
    input_variables=["context", "question", "option_1", "option_2", "option_3", "option_4", "option_5"],
    template="""
Use the following context to answer the multiple-choice question below.
Only answer with the correct option (A, B, C, D or E). Provide a short explanation.

[Context]
{context}

[Question]
{question}

[Options]
A. {option_1}
B. {option_2}
C. {option_3}
D. {option_4}
E. {option_5}

[Instruction]
The correct answer is
"""
)


rag_chain = RetrievalQA.from_chain_type(llm=llm, 
                                        retriever=retriever, 
                                        return_source_documents=False,
                                        input_key="question",
                                        chain_type_kwargs={"prompt": prompt_template}  
                                        )


/tmp/ipykernel_13751/4167135165.py:3: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  vectordb = Chroma(persist_directory=VECTOR_DB_PATH, embedding_function=HuggingFaceEmbeddings())
/tmp/ipykernel_13751/4167135165.py:3: LangChainDeprecationWarning: Default values for HuggingFaceEmbeddings.model_name were deprecated in LangChain 0.2.16 and will be removed in 0.4.0. Explicitly pass a model_name to the HuggingFaceEmbeddings constructor instead.
  vectordb = Chroma(persist_directory=VECTOR_DB_PATH, embedding_function=HuggingFaceEmbeddings())
/tmp/ipykernel_13751/4167135165.py:3: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9

## ORAN-Bench-13K Test Data Loading and Formatting

This script is responsible for loading multiple-choice questions (MCQs) from JSONL benchmark files and formatting them for use in LLM inference and evaluation.

- The `load_jsonl_mcq(path, sample_size, difficulty)` function:
  - Loads the question, options, and correct answer index.
  - Randomly samples a specified number of questions.
  - Converts each question into a dictionary containing `query`, `answer`, and `difficulty`.

- The `extract_predicted_option(text)` function is designed to extract answer choices (e.g., A/B/C/D or 1/2/3/4) from the model-generated output.
  - Example: `"The answer is C."` → `"C"`

- The loaded test samples are stored in `test_cases_all`. In the current setup, only `fin_E.json` is used with 50 questions.

✅ Test case loading is complete and ready for integration with LLM inference and accuracy evaluation workflows.


In [8]:
def load_jsonl_mcq(path, sample_size=100, difficulty=None):
    """從 JSONL 檔案載入選擇題格式資料，轉成 query-answer dict"""
    raw = []
    with open(path, "r", encoding="utf-8") as f:
        for line in f:
            raw.append(json.loads(line.strip()))

    sample_size = min(len(raw), sample_size)
    sampled = random.sample(raw, sample_size)

    formatted = []
    for item in sampled:
        question = item[0]
        options = item[1]
        answer_index = int(item[2]) - 1
        answer_text = options[answer_index]
        formatted.append({
            "query": question,
            "answer": answer_text,
            "difficulty": difficulty 
        })

    return formatted

def extract_predicted_option(text):
    # 嘗試從生成文字中抓出 A/B/C/D 或 1/2/3/4
    for opt in ["A", "B", "C", "D", "E", "1", "2", "3", "4"]:
        if f"{opt})" in text or f"{opt}." in text or f"The answer is {opt}" in text:
            return opt
    return "?"

# 評估工具
bleu = evaluate.load("bleu")
rouge = evaluate.load("rouge")
meteor = evaluate.load("meteor")

BENCHMARK_E_PATH = "branchmark/ORAN-Bench-13K/Benchmark/fin_E.json"
# BENCHMARK_M_PATH = "branchmark/ORAN-Bench-13K/Benchmark/fin_M.json"
# BENCHMARK_H_PATH = "branchmark/ORAN-Bench-13K/Benchmark/fin_H.json"

# 各取 100 題，並加上難度標記
test_cases_E = load_jsonl_mcq(BENCHMARK_E_PATH, sample_size=50, difficulty="E")
# test_cases_M = load_jsonl_mcq(BENCHMARK_M_PATH, sample_size=100, difficulty="M")
# test_cases_H = load_jsonl_mcq(BENCHMARK_H_PATH, sample_size=100, difficulty="H")

# 合併總測資（共 300 題）
# test_cases_all = test_cases_E + test_cases_M + test_cases_H
test_cases_all = test_cases_E
print(f"✅ 已建立 test_cases_all，共 {len(test_cases_all)} 題")



✅ 已建立 test_cases_all，共 50 題


[nltk_data] Downloading package wordnet to /home/aiml/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt_tab to /home/aiml/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /home/aiml/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


## TeleQnA

In [13]:
# Load question-answer pairs
with open("/home/aiml/johnson/thesis_rag/branchmark/TeleQnA/TeleQnA.txt", encoding="utf-8") as f:
    all_qa = json.load(f)

# 格式整理
qa_list = []
for item in all_qa.values():
    qa_list.append({
        "query": item["question"],
        "option 1": item.get("option 1", ""),
        "option 2": item.get("option 2", ""),
        "option 3": item.get("option 3", ""),
        "option 4": item.get("option 4", ""),
        "option 5": item.get("option 5", ""),
        "answer": item["answer"]
    })


# 顯示總題數
print(f"📊 Total questions loaded: {len(qa_list)}")

# 隨機挑選 100 題（不重複）
sample_size = 100
sampled_qa = random.sample(qa_list, min(sample_size, len(qa_list)))
# 可選：確認前 3 題內容
for i, qa in enumerate(sampled_qa[:3]):
    print(f"\n🧪 Sample {i+1}")
    print("Q:", qa["query"])
    print("A:", qa["answer"])
    
test_cases_all = sampled_qa
print(f"✅ 已隨機挑選 {len(sampled_qa)} 題測試題目")


📊 Total questions loaded: 10000

🧪 Sample 1
Q: What is the purpose of the fronthaul links in a CF (Cell-free) mMIMO (massive multiple-input-multiple-output) network?
A: option 1: To connect the APs to the CPUs

🧪 Sample 2
Q: What modulation format is also referred to as on-off keying (OOK)?
A: option 4: Amplitude shift keying (ASK)

🧪 Sample 3
Q: What are the two categories of channel estimation methods?
A: option 1: Training-based and non-training-based
✅ 已隨機挑選 100 題測試題目


## no_RAG

In [ ]:
print(f"✅ 開始執行 RAG 推論，共 {len(sampled_qa)} 題")
output_path = "/home/aiml/johnson/thesis_rag/output/no_rag_outputs.txt"

pipe = pipeline("text-generation", model=model, tokenizer=tokenizer, max_new_tokens=512, device=0)
llm = HuggingFacePipeline(pipeline=pipe)

hf_pipeline = llm.pipeline 

# 批次跑 non-RAG
batch_size = 10
no_rag_outputs = []
for i in tqdm(range(0, len(test_cases_all), batch_size)):
    batch_queries = [build_prompt_with_options(case) for case in sampled_qa]
    batch_outputs = llm.pipeline(batch_queries)

    for r in batch_outputs:
        if isinstance(r, list) and isinstance(r[0], dict) and "generated_text" in r[0]:
            no_rag_outputs.append(r[0]["generated_text"])
        elif isinstance(r, dict) and "generated_text" in r:
            no_rag_outputs.append(r["generated_text"])
        elif isinstance(r, str):
            no_rag_outputs.append(r)
        else:
            print("⚠️ Unexpected format:", r)
            no_rag_outputs.append("<<PARSE ERROR>>")

with open(output_path, "w", encoding="utf-8") as f:
    for line in no_rag_outputs:
        f.write(line + "\n")

print(f"✅ Non-RAG outputs saved to {output_path}")

Device set to use cuda:0


✅ 開始執行 RAG 推論，共 1 題


100%|██████████| 1/1 [00:00<00:00,  1.04it/s]

✅ Non-RAG outputs saved to /home/aiml/johnson/thesis_rag/output/no_rag_outputs.txt


## RAG

In [12]:
from tqdm import tqdm

rag_outputs = []
output_path_rag = "/home/aiml/johnson/thesis_rag/output/rag_outputs.txt"

print(f"✅ 開始執行 RAG 推論，共 {len(sampled_qa)} 題")
print(sampled_qa[0])

for i in tqdm(range(len(sampled_qa))):
    case = sampled_qa[i]
    input_dict = {
    "question": case["query"],
    **{f"option_{j}": case.get(f"option {j}", "") for j in range(1, 6)}
}
    

    try:
        result = rag_chain.run(input_dict)
        rag_outputs.append(result)
    except Exception as e:
        print(f"❌ Error on case {i}: {e}")
        rag_outputs.append("<<ERROR>>")

# 儲存到檔案
with open(output_path_rag, "w", encoding="utf-8") as f:
    for line in rag_outputs:
        f.write(line.strip() + "\n")

print(f"✅ RAG outputs saved to {output_path_rag}")


✅ 開始執行 RAG 推論，共 1 題
{'query': 'What is the responsibility of the network slice broker (NSB)?', 'option 1': 'To orchestrate and manage the US-NSIs', 'option 2': 'To securely expose required services and capabilities to every US-NSI', 'option 3': 'To dynamically allocate network resources to every US-NSI', 'option 4': "To translate tenant requirements to US-NSIs' requirements", 'option 5': 'To enable the tenant to dynamically request and lease resources from the operator', 'answer': 'option 5: To enable the tenant to dynamically request and lease resources from the operator'}


100%|██████████| 1/1 [00:00<00:00, 26.54it/s]

❌ Error on case 0: Missing some input keys: {'option_4', 'option_2', 'option_3', 'option_5', 'option_1'}
✅ RAG outputs saved to /home/aiml/johnson/thesis_rag/output/rag_outputs.txt


In [26]:
from evaluate import load
bleu = load("bleu")
rouge = load("rouge")
meteor = load("meteor")

# 預備資料
references = [case["answer"] for case in test_cases_all]

# 批次評分
bleu_norag = bleu.compute(predictions=no_rag_outputs, references=[[ref] for ref in references])["bleu"]
bleu_rag = bleu.compute(predictions=rag_outputs, references=[[ref] for ref in references])["bleu"]

rouge_norag = rouge.compute(predictions=no_rag_outputs, references=references)["rougeL"]
rouge_rag = rouge.compute(predictions=rag_outputs, references=references)["rougeL"]

meteor_norag = meteor.compute(predictions=no_rag_outputs, references=references)["meteor"]
meteor_rag = meteor.compute(predictions=rag_outputs, references=references)["meteor"]


[nltk_data] Downloading package wordnet to /home/aiml/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt_tab to /home/aiml/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /home/aiml/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [28]:
print("📊 BLEU")
print(f"  No RAG: {bleu_norag:.4f}")
print(f"  RAG   : {bleu_rag:.4f}")
print("📊 ROUGE-L")
print(f"  No RAG: {rouge_norag:.4f}")
print(f"  RAG   : {rouge_rag:.4f}")
print("📊 METEOR")
print(f"  No RAG: {meteor_norag:.4f}")
print(f"  RAG   : {meteor_rag:.4f}")


📊 BLEU
  No RAG: 0.0010
  RAG   : 0.0011
📊 ROUGE-L
  No RAG: 0.0182
  RAG   : 0.0255
📊 METEOR
  No RAG: 0.0460
  RAG   : 0.0620


In [29]:
acc_norag = 0
acc_rag = 0

for i, case in enumerate(test_cases_all):
    gold = case["answer"]
    pred_no_rag = extract_predicted_option(no_rag_outputs[i])
    pred_rag = extract_predicted_option(rag_outputs[i])

    if pred_no_rag in gold:
        acc_norag += 1
    if pred_rag in gold:
        acc_rag += 1

acc_norag /= len(test_cases_all)
acc_rag /= len(test_cases_all)

print(f"✅ Accuracy (No RAG): {acc_norag:.2%}")
print(f"✅ Accuracy (RAG): {acc_rag:.2%}")


✅ Accuracy (No RAG): 10.00%
✅ Accuracy (RAG): 3.00%
